In [2]:
from xml.dom import minidom
import simplejson as json
def parse_element(element):
    dict_data = dict()
    if element.nodeType == element.TEXT_NODE:
        if element.data != " ":
            dict_data['data'] = element.data
    if element.nodeType not in [element.TEXT_NODE, element.DOCUMENT_NODE, 
                                element.DOCUMENT_TYPE_NODE]:
        for item in element.attributes.items():
            dict_data[item[0]] = item[1]
    if element.nodeType not in [element.TEXT_NODE, element.DOCUMENT_TYPE_NODE]:
        for child in element.childNodes:
            child_name, child_dict = parse_element(child)
            if child_name in dict_data:
                try:
                    dict_data[child_name].append(child_dict)
                except AttributeError:
                    dict_data[child_name] = [dict_data[child_name], child_dict]
            else:
                dict_data[child_name] = child_dict 
    return element.nodeName, dict_data

if __name__ == '__main__':
    dom = minidom.parse('cqa_task3.xml')
    print type(dom)
    f = open('cqa_task3.json', 'w')
    f.write(json.dumps(parse_element(dom), sort_keys=True, indent=4))
    f.close()

<type 'instance'>


In [3]:
import pandas as pd

data1 = pd.read_json('cqa_task3.json')
data1.head()
df31 = pd.io.json.json_normalize(data1[0][1]['xml'][1]['Thread'])
df41 = pd.concat([pd.DataFrame(pd.io.json.json_normalize(x)) for x in df31['RelComment']],ignore_index=True)
df41['THREAD_SEQUENCE'] = [ '_'.join(i.split('_')[:2]) for i in df41['RELC_ID']]
df51 = pd.concat([df31.set_index('THREAD_SEQUENCE'),df41.set_index('THREAD_SEQUENCE')], axis=1, join='inner').reset_index()
df51.head()
rem_cols = ['RelQuestion.#text' , 'RelComment', 'RelCBody.#text.data' , 'RelQuestion.RelQBody.#text.data' , 
            'RelQuestion.RelQBody.#text.data', '#text']
[ df51.drop(i, axis=1, inplace=True) for i in rem_cols  ]
df51.head()
#df51[ df51['RelQuestion.RelQSubject.#text.data'] == 'Best Bank.' ]

,THREAD_SEQUENCE,RelQuestion.RELQ_CATEGORY,RelQuestion.RELQ_DATE,RelQuestion.RELQ_ID,RelQuestion.RELQ_USERID,RelQuestion.RELQ_USERNAME,RelQuestion.RelQClean.#text.data,RelQuestion.RelQSubject.#text.data,RELC_DATE,RELC_ID,RELC_RELEVANCE2RELQ,RELC_USERID,RELC_USERNAME,RelCClean.#text.data
0,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-07-31 06:46:39,Q268_R16_C1,Bad,U65,Molten Metal,banks are using us ... Talk to those who had t...
1,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-07-31 08:10:53,Q268_R16_C2,Bad,U956,Rip Cord,In Qatar that is like saying which is the best...
2,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-07-31 10:37:03,Q268_R16_C3,Bad,U5152,arman1arzoo,I'm surprised to see such feedbacks on Qatar b...
3,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-08-01 11:38:21,Q268_R16_C4,Good,U5153,westernindoha,Well Arman; nothing is wrong here with banks; ...
4,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-08-01 11:45:49,Q268_R16_C5,Good,U492,happygolucky,With QNB for last 4 years plus...no issues...g...
